In [1]:
import pandas as pd
import numpy as np
import dash
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

In [2]:
#external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
external_stylesheets = [dbc.themes.MINTY]

## Initialize the app

In [3]:
app = JupyterDash(__name__, external_stylesheets=external_stylesheets) # Jupyter version
#app = dash.Dash(__name__, external_stylesheets=external_stylesheets) # regular version

## Specify the contents of the dashboard

In [4]:
app.layout = html.Div(
[
    html.H1("Clueless Dashboard"),
    
    dcc.Input(id = 'username', type = 'text', placeholder = 'Please type your name here.'),

    dcc.Markdown(id = 'countletters')
]
)

## Callbacks

In [10]:
@app.callback(Output(component_id = 'countletters', component_property = 'children'), 
              Input(component_id = 'username', component_property = 'value'))

def countletters(name):
    n = str(name).replace(' ', '')
    return f"You have {len(n)} letters in your name!"

In [11]:
countletters('Meesun Y')

'You have 7 letters in your name!'

## Run the app

In [7]:
#!pip install python-dotenv

In [8]:
# NEED TO FIX THIS LATER
if __name__== "__main__":
    app.run_server(mode= 'inline', host = '0.0.0.0', debug=True)